# This is the API documentation for Notes-api

Theres several types of main routes:
```
Public routes: Notes routes without the need of authorization
Auth routes: Authorization flow (run this before running Notes routes)
User routes: Some user related configs
Notes routes: The crud application with authorization
```

### Prerequisites
⬇️️️ Please run this block below before running everything else. ⬇️

In [146]:
%use ktor-client
import kotlinx.serialization.Serializable
import java.util.UUID

//Ktor Client initialization

import io.ktor.client.HttpClient
import io.ktor.client.plugins.contentnegotiation.ContentNegotiation
import io.ktor.client.request.get
import io.ktor.serialization.kotlinx.json.json

val baseUrl = "http://127.0.0.1:8081"

@Serializable
data class Note(
    val id: String? = UUID.randomUUID().toString(),
    val title: String?,
    val content: String?,
    val createdAt: Long? = System.currentTimeMillis(),
    val updatedAt: Long? = null
)

@Serializable
data class NeoUser(
    val id: String? = UUID.randomUUID().toString(),
    val username: String,
    val email: String,
    val password: String,
    val createdAt: Long? = System.currentTimeMillis()
)

fun HttpResponse.prettier(): String {
    val prettyJson = Json { prettyPrint = true }
    val lol = Json.parseToJsonElement(this.deserializeJson().toString())
    return prettyJson.encodeToString(lol)
}

val ktorClient = HttpClient() {
    install(ContentNegotiation) {
        json()
    }
}

## Public Routes (Without authorization)
```
|-get notes
|-create note
|-update note
|-delete note
```

⬇️️️ Please run this block below before running any routes from Public routes. ⬇️

In [147]:
import io.ktor.client.call.body
import io.ktor.client.request.post
import io.ktor.client.request.setBody
import io.ktor.http.ContentType
import io.ktor.http.contentType
import io.ktor.http.headers
import io.ktor.client.request.headers
import io.ktor.client.request.put
import io.ktor.client.request.setBody
import io.ktor.client.request.delete

In [148]:
//get notes
runBlocking {
    val notes = ktorClient.get("$baseUrl/notes")
    print(notes.prettier())
}

[
    {
        "id": "433a1af4-bf84-485f-bb78-72e8b66873c1",
        "userId": "null",
        "title": "Legend",
        "content": "hahahahah",
        "createdAt": 1753873160151,
        "updatedAt": 1753873181963
    },
    {
        "id": "eaafdb26-f7f1-4de5-94a1-0723cbf74ad3",
        "userId": "null",
        "title": "Two",
        "content": "Content two",
        "createdAt": 1753884344531,
        "updatedAt": 1753884344580
    },
    {
        "id": "e266ac44-0757-4e69-906e-c4ab9ac86607",
        "userId": "null",
        "title": "Four",
        "content": "Content four",
        "createdAt": 1753889994091,
        "updatedAt": 1753889994111
    },
    {
        "id": "b20fa5b3-16b7-47c7-bdbd-8982fb41db11",
        "userId": "null",
        "title": "Five",
        "content": "Content five",
        "createdAt": 1754065428562,
        "updatedAt": 1754065428711
    },
    {
        "id": "61c2de5f-b556-4985-b8d2-3b111fe55162",
        "userId": "null",
        "title": "F

In [144]:
//create note

runBlocking {
    val newNote = Note(
        title = "7",
        content = "Content 7"
    )
    val postNote = ktorClient.post("$baseUrl/notes"){
        headers {
            contentType(ContentType.Application.Json)
        }
        setBody(newNote)
    }
    println("response: $postNote")
    println("data: ${postNote.deserializeJson()}")
}

response: HttpResponse[http://127.0.0.1:8081/notes, 200 OK]
data: {"id":"0f71468a-99af-40c8-a613-79a2f8595ff2","userId":"null","title":"7","content":"Content 7","createdAt":1754231649881,"updatedAt":1754231649941}


In [153]:
//update note
runBlocking {
    val id = "536eb286-b325-49e1-9934-0d13d154d02f"
    val updatedNote = Note(
        title = "The Alchemist!!",
        content = "Paulo Coelho wrote this book."
    )

    val result = ktorClient.put("$baseUrl/notes/$id") {
        headers {
            contentType(ContentType.Application.Json)
        }
        setBody(updatedNote)
    }

    println("response: $result")
    println("data: ${result.prettier()}")
}

response: HttpResponse[http://127.0.0.1:8081/notes/0f71468a-99af-40c8-a613-79a2f8595ff2, 200 OK]
data: {
    "id": "0f71468a-99af-40c8-a613-79a2f8595ff2",
    "userId": "null",
    "title": "The Alchemist!!",
    "content": "Paulo Coelho wrote this book.",
    "createdAt": 1754231649881,
    "updatedAt": 1754232080224
}


In [155]:
//delete note
runBlocking {
    val id = "bacf46e2-b520-4728-b4ce-8a31158fd179"
    val deleteNote = ktorClient.delete("$baseUrl/notes/$id")

    println("response: $deleteNote")
    println("data: ${deleteNote.deserializeJson()}")
}

response: HttpResponse[http://127.0.0.1:8081/notes/bacf46e2-b520-4728-b4ce-8a31158fd179, 200 OK]
data: Note deleted.


## Auth Routes
```
|-login
|-register
|-check username
|-delete user by id
|-get all users
|-check user
```

⬇️️️ Please run this block below before running any routes from Public routes. ⬇️

In [ ]:
//Ktor Client initialization

import io.ktor.client.HttpClient
import io.ktor.client.engine.cio.CIO
import io.ktor.client.plugins.auth.Auth
import io.ktor.client.plugins.auth.providers.BasicAuthCredentials
import io.ktor.client.plugins.auth.providers.basic
import io.ktor.client.plugins.contentnegotiation.ContentNegotiation
import io.ktor.client.request.get
import io.ktor.serialization.kotlinx.json.json
import io.ktor.client.call.body

In [156]:
//login user
@Serializable
data class TokenData(
    val token: String? = null
)

var token = TokenData()
fun login() {
    runBlocking {
        val response = ktorClient.get("$baseUrl/auth") {
            headers.append("Username", "martin3")
            headers.append("Password", "martin1")
        }.body<TokenData>()
        token = response
    }
}
//println("response: ${token}")

login()

val tempp = token.token

In [ ]:
//register a user
//import Line_146_jupyter.NeoUser

runBlocking {
    val newUser = NeoUser(
        username = "martin3",
        password = "martin1",
        email = "martin3@g.com"
    )
    val response = ktorClient.post("$baseUrl/auth") {
        headers {
            contentType(ContentType.Application.Json)
        }
        setBody(newUser)
    }
    println("response: $response")
    println("data: ${response.deserializeJson()}")
}

In [ ]:
//check username
runBlocking {
    val username = ktorClient.get("$baseUrl/user/username") {
        headers.append(HttpHeaders.Authorization, "Bearer $tempp")
    }

    println("response: $username")
    println("data: ${username.deserializeJson()}")
}

In [ ]:
// delete a user by id
import io.ktor.client.request.headers

runBlocking {
    val response = ktorClient.delete("$baseUrl/auth") {
        headers.append("Id", "1b677ac8-ad8b-4da4-a29f-6720738f1b5f")
    }
    println("response: $response")
    println("data: ${response.deserializeJson()}")
}

In [ ]:
//get all users
runBlocking {
    val response = ktorClient.get("$baseUrl/auth/users")
    println("response: $response")
    println("data: ${response.deserializeJson()}")
}

In [ ]:
//check user
runBlocking {
    val response = ktorClient.get("$baseUrl/auth/check") {
        headers.append("Username", "martin3")
    }
    println("response: $response")
    println("data: ${response.deserializeJson()}")
}

## User Routes
```
|-get user ID
|-get username
```

⬇️️️ Please run this block below before running any routes from Public routes. ⬇️

In [169]:
//check user
runBlocking {
    val response = ktorClient.get("$baseUrl/user/userid") {
        headers.append(HttpHeaders.Authorization, "Bearer $tempp")
    }
    println("response: $response")
    println("data: ${response.deserializeJson()}")
}

response: HttpResponse[http://127.0.0.1:8081/user/userid, 200 OK]
data: f440ced9-cebc-44f0-b2d5-e7bf11211daf


In [167]:
//get username
runBlocking {
    val response = ktorClient.get("$baseUrl/user/username") {
        headers.append(HttpHeaders.Authorization, "Bearer $tempp")
    }
    println("response: $response")
    println("data: ${response.deserializeJson()}")
}

response: HttpResponse[http://127.0.0.1:8081/user/username, 200 OK]
data: martin3


## Notes Routes
```
|-get notes
|-create note
|-update note
|-delete note
```

⬇️️️ Please run this block below before running any routes from Public routes. ⬇️

In [ ]:
import io.ktor.client.call.body
import io.ktor.client.request.post
import io.ktor.client.request.setBody
import io.ktor.http.ContentType
import io.ktor.http.contentType
import io.ktor.http.headers

In [159]:
//get notes
login()
runBlocking {
    val notes = ktorClient.get("$baseUrl/user/notes") {
        headers.append(HttpHeaders.Authorization, "Bearer $tempp")
    }.prettier()
    print(notes)
}

[
    {
        "id": "f0e7f38f-bcbd-4441-93b2-055a407774ac",
        "userId": "f440ced9-cebc-44f0-b2d5-e7bf11211daf",
        "title": "3",
        "content": "Content 3",
        "createdAt": 1754212588329,
        "updatedAt": 1754212588365
    },
    {
        "id": "3f32f79c-4233-4842-a165-5dcc8fa822dc",
        "userId": "f440ced9-cebc-44f0-b2d5-e7bf11211daf",
        "title": "4",
        "content": "Content 4",
        "createdAt": 1754212607654,
        "updatedAt": 1754212607668
    },
    {
        "id": "385831a4-28d4-4a9b-b878-96fca8d09915",
        "userId": "f440ced9-cebc-44f0-b2d5-e7bf11211daf",
        "title": "6",
        "content": "Content 6",
        "createdAt": 1754232567538,
        "updatedAt": 1754232567584
    }
]

In [158]:
//create note
login()
runBlocking {
    val newNote = Note(
        title = "6",
        content = "Content 6"
    )
    val postNote = ktorClient.post("$baseUrl/user/notes"){
        headers {
            headers.append(HttpHeaders.Authorization, "Bearer $tempp")
            contentType(ContentType.Application.Json)
        }
        setBody(newNote)
    }.prettier()
    println("response: $postNote")
    println("data: ${postNote.deserializeJson()}")
}

response: {
    "id": "385831a4-28d4-4a9b-b878-96fca8d09915",
    "userId": "f440ced9-cebc-44f0-b2d5-e7bf11211daf",
    "title": "6",
    "content": "Content 6",
    "createdAt": 1754232567538,
    "updatedAt": 1754232567584
}
data: {
    "id": "385831a4-28d4-4a9b-b878-96fca8d09915",
    "userId": "f440ced9-cebc-44f0-b2d5-e7bf11211daf",
    "title": "6",
    "content": "Content 6",
    "createdAt": 1754232567538,
    "updatedAt": 1754232567584
}


In [160]:
//update note
runBlocking {
    val id = "3f32f79c-4233-4842-a165-5dcc8fa822dc"
    val updatedNote = Note(
        title = "The Tell Tale Hearts!!",
        content = "Edgar Allan Poe wrote this."
    )

    val result = ktorClient.put("$baseUrl/user/notes/$id") {
        headers {
            headers.append(HttpHeaders.Authorization, "Bearer $tempp")
            contentType(ContentType.Application.Json)
        }
        setBody(updatedNote)
    }

    println("response: $result")
    println("data: ${result.prettier()}")
}

response: HttpResponse[http://127.0.0.1:8081/user/notes/3f32f79c-4233-4842-a165-5dcc8fa822dc, 200 OK]
data: {
    "id": "3f32f79c-4233-4842-a165-5dcc8fa822dc",
    "userId": "null",
    "title": "The Tell Tale Hearts!!",
    "content": "Edgar Allan Poe wrote this.",
    "createdAt": 1754212607654,
    "updatedAt": 1754232643443
}


In [161]:
//delete note
runBlocking {
    val id = "3f32f79c-4233-4842-a165-5dcc8fa822dc"
    val deleteNote = ktorClient.delete("$baseUrl/user/notes/$id") {
        headers.append(HttpHeaders.Authorization, "Bearer $tempp")
    }

    println("response: $deleteNote")
    println("data: ${deleteNote.deserializeJson()}")
}

response: HttpResponse[http://127.0.0.1:8081/user/notes/3f32f79c-4233-4842-a165-5dcc8fa822dc, 200 OK]
data: Note deleted.
